## Create a permanent forest cover layer for Landsat data using NLCD 

In [20]:
import ee
import geemap 
import os
ee.Initialize()

roi = ee.FeatureCollection('users/kmcquil/SBR_XL_simplified').geometry()


In [5]:
# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')

# The collection contains images for multiple years and regions in the USA.
print('Products:', dataset.aggregate_array('system:index').getInfo())

# have one for 1992, 2001, 2004, 2006, 2008, 2011, 2013, 2016
# 1992 just has land cover 

# Filter the collection to the 2016 product.
nlcd2016 = dataset.filter(ee.Filter.eq('system:index', '1992')).first()

# Each product has multiple bands for describing aspects of land cover.
print('Bands:', nlcd2016.bandNames().getInfo())

# Select the land cover band.
#landcover = nlcd2016.select('landcover')


Products: ['1992', '2001', '2001_AK', '2001_HI', '2001_PR', '2004', '2006', '2008', '2011', '2011_AK', '2011_HI', '2011_PR', '2013', '2016', '2016_AK', '2016_HI', '2016_PR']
Bands: ['landcover']


In [ ]:
# for each year
# select land cover 
# create a mask to look just at the forested land cover types 

In [35]:
# forest is 41, 42, 43 
def LC(y):
    nlcd = dataset.filter(ee.Filter.eq('system:index', y))
    landcover = nlcd.select('landcover').mosaic().clip(roi)
    return landcover.eq(41).max(landcover.eq(42)).max(landcover.eq(43))



In [36]:
# apply mask to all years 
years = ee.List(['1992', '2001', '2004', '2006', '2008', '2011', '2013', '2016'])
lc_mask_years = ee.ImageCollection.fromImages(years.map(LC).flatten())

In [37]:
count=lc_mask_years.size()
print('count =', str(count.getInfo())+'\n')

count = 8



In [38]:
# sum all of the years 
lc_sum = lc_mask_years.sum()

In [44]:
# there are 8 years, so we only want pixels = 8 
lc_sum8 = lc_sum.eq(8)

lc_sum8 = lc_sum8.updateMask(lc_sum8)

In [45]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [46]:
igbpLandCoverVis = {
  min: 8.0,
  max: 19.0,
  'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301' ],
}
Map.addLayer(lc_sum8, igbpLandCoverVis)

In [47]:
# export the permanent forest cover layer 

task_config = {
        'region': roi,
        'fileFormat': 'GeoTIFF',
        'folder':'LANDSAT_FOREST',
        'description': 'landsat_permanent_forest',
        'image': lc_sum8,
        'maxPixels':1e13, 
        'scale' : 30
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()